In [342]:
import numpy as np
import cupy as cp
from cuquantum import custatevec as cusv
from cuquantum import cudaDataType as cudtype
from cuquantum import ComputeType as ctype
import matplotlib.pyplot as plt
import time

# 应用 Oracle 的函数
def apply_oracle(handle, d_sv, target_index):
    """
    为 Grover 算法应用 Oracle。
    翻转由 target_index 指定的目标态的符号。
    """
    # 翻转目标态的相位
    d_sv[target_index] *= -1

# 使用反射矩阵应用扩散算子的函数
def apply_diffusion_operator(handle, d_sv, nIndexBits):
    """
    为 Grover 算法应用扩散算子。
    该操作等效于 2|ψ⟩⟨ψ| - I。
    """
    # 创建用于反射的均匀叠加态向量
    uniform_amplitude = 1.0 / np.sqrt(len(d_sv))
    d_superposition = cp.full_like(d_sv, uniform_amplitude, dtype=np.complex128)

    # 关于均匀叠加态应用反射
    dot_product = cp.vdot(d_superposition, d_sv)  # 内积 <ψ|s>
    d_sv[:] = 2 * dot_product * d_superposition - d_sv

    # 归一化状态向量以确保概率和为 1
    norm = cp.linalg.norm(d_sv)
    d_sv /= norm

# Grover 算法主循环
def grover_algorithm(handle, d_sv, nIndexBits, target_index, nIter=2):
    """
    执行指定迭代次数的 Grover 算法。
    """
    for i in range(nIter):
        # 应用 Oracle
        apply_oracle(handle, d_sv, target_index)

        # 应用扩散算子
        apply_diffusion_operator(handle, d_sv, nIndexBits)

        # 概率分布可视化
        probabilities = cp.abs(d_sv) ** 2
        probabilities = cp.asnumpy(probabilities)

# 初始化量子比特和状态向量
nIndexBits = 3  # 量子比特数量
nSvSize = (1 << nIndexBits)  # 状态向量的大小（2^nIndexBits）
d_sv = cp.full((nSvSize,), 1.0 / np.sqrt(nSvSize), dtype=np.complex128)  # 初始化为均匀叠加态
start = time.time()
# cuStateVec 句柄初始化
handle = cusv.create()

# 定义 Oracle 的目标索引（例如，|110> 对应于索引 6）
target_index = 6
# 根据系统的规模和目标态数量，计算最佳迭代次数
N = 1 << nIndexBits  # 总状态数，等于 2^nIndexBits
M = 1  # 目标态数量 (这里假设只有一个目标态)
optimal_iterations = int(np.round(np.pi / 4 * np.sqrt(N / M)))

# 使用最佳迭代次数执行 Grover 算法以获得更好结果
grover_algorithm(handle, d_sv, nIndexBits, target_index, nIter=optimal_iterations)

# 检查 Grover 算法后的状态向量
final_state_vector = cp.asnumpy(d_sv)
# print("Final state vector:", final_state_vector)

# 计算保真度
ideal_state = np.zeros(nSvSize, dtype=np.complex128)
ideal_state[target_index] = 1.0
fidelity = np.abs(np.vdot(ideal_state, final_state_vector)) ** 2
print(f"Fidelity with target state: {fidelity}")
end = time.time()

print (str(end-start))
# # 绘制最终的概率分布
# probabilities = np.abs(final_state_vector) ** 2
# plt.bar(range(len(probabilities)), probabilities)
# plt.xlabel('Basis State Index')
# plt.ylabel('Probability')
# plt.title('Final Probability Distribution')
# plt.show()

# 清理
cusv.destroy(handle)


Fidelity with target state: 0.9999997569653615
3.3221933841705322


In [450]:
from qiskit_aer import Aer
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import plot_histogram
import numpy as np
import matplotlib.pyplot as plt
from qiskit.quantum_info import Statevector, state_fidelity
import time
# 定义 Grover 算法的 Qiskit 电路
def grover_circuit(num_qubits, target_state, num_iterations):
    """
    构建 Grover 算法的量子电路。
    :param num_qubits: 量子比特的数量
    :param target_state: 目标状态（需要找到的状态）
    :param num_iterations: Grover 算法的迭代次数
    :return: QuantumCircuit 对象
    """
    # 初始化量子电路
    qc = QuantumCircuit(num_qubits)

    # 将所有量子比特初始化为均匀叠加态
    qc.h(range(num_qubits))

    # Grover 迭代过程
    for _ in range(num_iterations):
        # Oracle: 对目标态进行相位翻转
        for qubit in range(num_qubits):
            if target_state & (1 << qubit) == 0:
                qc.x(qubit)
        qc.h(num_qubits - 1)
        qc.mcx(list(range(num_qubits - 1)), num_qubits - 1)  # 多控制 Toffoli 门
        qc.h(num_qubits - 1)
        for qubit in range(num_qubits):
            if target_state & (1 << qubit) == 0:
                qc.x(qubit)

        # Diffusion operator: 扩散算子
        qc.h(range(num_qubits))
        qc.x(range(num_qubits))
        qc.h(num_qubits - 1)
        qc.mcx(list(range(num_qubits - 1)), num_qubits - 1)
        qc.h(num_qubits - 1)
        qc.x(range(num_qubits))
        qc.h(range(num_qubits))

    return qc

# 参数设置
num_qubits = 22  # 量子比特数量
target_state = 6  # 目标态（例如 |110> 对应索引 6）
num_iterations = int(np.round(np.pi / 4 * np.sqrt(2**num_qubits)))  # 根据系统规模计算最佳迭代次数
start = time.time()
# 构建 Grover 电路
qc = grover_circuit(num_qubits, target_state, num_iterations)
qc.measure_all()

# 使用 Qiskit Aer 后端进行仿真
simulator = Aer.get_backend('qasm_simulator')
compiled_circuit = transpile(qc, simulator)
result = simulator.run(compiled_circuit).result()
counts = result.get_counts()

# 打印测量结果
print("Measurement results:", counts)

# 绘制测量结果的柱状图
plot_histogram(counts)
plt.show()

# 使用状态向量模拟器来获取仿真状态向量
qc_no_measure = grover_circuit(num_qubits, target_state, num_iterations)

simulator = Aer.get_backend('statevector_simulator')
compiled_circuit = transpile(qc_no_measure, simulator)
job = simulator.run(compiled_circuit)
result = job.result()
simulated_statevector = result.get_statevector()

# 目标态的状态向量
target_vector = np.zeros(2**num_qubits)
target_vector[target_state] = 1
target_statevector = Statevector(target_vector)

# 计算保真度
end = time.time()

print (str(end-start))
fidelity = state_fidelity(target_statevector, simulated_statevector)
print(f"Fidelity with target state: {fidelity}")

# # 绘制最终的概率分布
# probabilities = np.abs(simulated_statevector.data) ** 2
# plt.bar(range(len(probabilities)), probabilities)
# plt.xlabel('Basis State Index')
# plt.ylabel('Probability')
# plt.title('Final Probability Distribution')
# plt.show()


Measurement results: {'0000000000000000000110': 1024}
667.1229107379913
Fidelity with target state: 0.9999999999796017
